In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from collections import defaultdict
import argparse
import cv2  # NOQA (Must import before importing caffe2 due to bug in cv2)
import glob
import logging
import os
import sys
import time

from caffe2.python import workspace

from detectron.core.config import assert_and_infer_cfg
from detectron.core.config import cfg
from detectron.core.config import merge_cfg_from_file
from detectron.utils.io import cache_url
from detectron.utils.logging import setup_logging
from detectron.utils.timer import Timer
import detectron.core.test_engine as infer_engine
import detectron.datasets.dummy_datasets as dummy_datasets
import detectron.utils.c2 as c2_utils
import detectron.utils.vis as vis_utils

In [2]:
c2_utils.import_detectron_ops()
cv2.ocl.setUseOpenCL(False)
merge_cfg_from_file("../configs/DensePose_ResNet101_FPN_s1x-e2e.yaml")
cfg.NUM_GPUS = 1
model = infer_engine.initialize_model_from_cfg("../DensePoseData/DensePose_ResNet101_FPN_s1x-e2e.pkl")
workspace.GlobalInit(['caffe2', '--caffe2_log_level=0'])

Found Detectron ops lib: /usr/local/lib/libcaffe2_detectron_ops_gpu.so


In [3]:
timers = defaultdict(Timer)
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    img = frame.copy()
    with c2_utils.NamedCudaScope(0):
        cls_boxes, cls_segms, cls_keyps, cls_bodys = infer_engine.im_detect_all(
            model, img, None, timers=timers
        )
    vis_utils.vis_one_image_opencv(
        frame,
        cls_boxes,
        cls_segms,
        cls_keyps,
        thresh=0.7,
        show_class=True
    )
    cv2.imshow('frame', frame)
    if cv2.waitKey(120) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

RuntimeError: [enforce fail at blob.h:84] IsType<T>(). wrong type for the Blob instance. Blob contains nullptr (uninitialized) while caller expects caffe2::Tensor<caffe2::CUDAContext> .
Offending Blob name: gpu_0/rois_fpn5.
Error from operator: 
input: "gpu_0/fpn_res5_2_sum" input: "gpu_0/rois_fpn5" output: "gpu_0/roi_feat_fpn5" name: "" type: "RoIAlign" arg { name: "pooled_h" i: 7 } arg { name: "sampling_ratio" i: 2 } arg { name: "spatial_scale" f: 0.03125 } arg { name: "pooled_w" i: 7 } device_option { device_type: 1 cuda_gpu_id: 0 }